#### Reinforcement Learning and Decision Making &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Homework #1

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Planning in MDPs

## Description

You are given an $N$-sided die, along with a corresponding Boolean mask
vector, `is_bad_side` (i.e., a vector of ones and zeros). You can assume
that $1<N\leq30$, and the vector `is_bad_side` is also of size $N$ and
$1$ indexed (since there is no $0$ side on the die). The game of DieN is
played as follows:

1.  You start with $0$ dollars.

2.  At any time you have the option to roll the die or to quit the game.

    1.  **ROLL**:

        1.  If you roll a number not in `is_bad_side`, you receive that
            many dollars (e.g., if you roll the number $2$ and $2$ is
            not a bad side -- meaning the second element of the vector
            `is_bad_side` is $0$, then you receive $2$ dollars). Repeat
            step 2.

        2.  If you roll a number in `is_bad_side`, then you lose all the
            money obtained in previous rolls and the game ends.

    2.  **QUIT**:

        1.  You keep all the money gained from previous rolls and the
            game ends.

## Procedure

-   You will implement your solution using the `solve()` method
    in the code below.
    
-   Your return value should be the number of dollars you expect to
    win for a specific value of `is_bad_side`, if you follow an
    optimal policy. That is, what is the value of the optimal
    state-value function for the initial state of the game (starting
    with $0$ dollars)? Your answer must be correct to $3$ decimal
    places, truncated (e.g., 3.14159265 becomes 3.141).

-   To solve this problem, you will need to determine an optimal policy
    for the game of DieN, given a particular configuration of the die.
    As you will see, the action that is optimal will depend on your
    current bankroll (i.e., how much money you've won so far).

-   You can try solving this problem by creating an MDP of the game
    (states, actions, transition function, reward function, and assume a
    discount rate of $\gamma=1$) and then calculating the optimal
    state-value function.

## Resources

The concepts explored in this homework are covered by:

-   Lecture Lesson 1: Smoov & Curly's Bogus Journey

-   Chapter 3 (3.6 Optimal Policies and Optimal Value Functions) and
    Chapter 4 (4.3-4.4 Policy Iteration, Value Iteration) of
    http://incompleteideas.net/book/the-book-2nd.html

-   Chapters 1-2 of 'Algorithms for Sequential Decision Making', M.
    Littman, 1996

## Submission

-   The due date is indicated on the Canvas page for this assignment.
    Make sure you have your timezone in Canvas set to ensure the
    deadline is accurate.

-   Submit your finished notebook on Gradescope. Your grade is based on
    a set of hidden test cases. You will have unlimited submissions -
    only the last score is kept.

-   Use the template below to implement your code. We have also provided
    some test cases for you. If your code passes the given test cases,
    it will run (though possibly not pass all the tests) on Gradescope.

-   Gradescope is using *python 3.6.x* and *numpy==1.18.0*, and you can
    use any core library (i.e., anything in the Python standard library).
    No other library can be used.  Also, make sure the name of your
    notebook matches the name of the provided notebook.  Gradescope times
    out after 10 minutes.

In [8]:
#################
# DO NOT REMOVE
# Versions
# numpy==1.18.0
################
import numpy as np
class MDPAgent(object):
    def __init__(self):
        pass
    
    def set_environment(self, face_count, state_count, good_sides):
        T = np.zeros((2, state_count, state_count))
        R = np.zeros((2, state_count, state_count))
        
        possible_bankroll_state = good_sides.copy()
        possible_bankroll_state.insert(0,0)
        
        T_roll = np.zeros((state_count, state_count))
        T_quit = np.zeros((state_count, state_count))
        R_roll = np.zeros((state_count, state_count))
        R_quit = np.zeros((state_count, state_count))
        
        # loop through all state values and set T/R       
        for state_val in range(0, state_count):
            T_roll_val = np.zeros(state_count)
            R_roll_val = np.zeros(state_count)
            # if the state value is in the array of possible bankroll states, we need to update 
            if state_val in possible_bankroll_state:
                p_t = 1
                # go through each possible roll
                for side in good_sides:
                    # generate a new state by adding them and if it's within the cap
                    new_state = side + state_val
                    if new_state < state_count - 1:
                        # update the transition probability for that new state                        
                        p_t = p_t - (1 / face_count)
                        T_roll_val[new_state] = 1 / face_count
                        # update the reward for that new state 
                        R_roll_val[new_state] = side
                        # update the state to include new bankroll val
                        if new_state not in possible_bankroll_state:
                            possible_bankroll_state.append(new_state)
                T_roll_val[state_count - 1] = p_t
                R_roll_val[state_count - 1] = -state_val
            
            T_roll[state_val] = T_roll_val
            R_roll[state_val] = R_roll_val
            
        T = [T_roll, T_quit]
        R = [R_roll, R_quit]
        return T, R
    
    def determine_initial_state_value(self, T, probabilities, state_count):
        values = np.empty(state_count)
        policy = np.empty((2, state_count))
        i = 0
        max_iter = 1000000000
        while True:
            i += 1
            prev_vals = values.copy()
            # calculate policy for roll, quit and choose the action with the highest value 
            for i, _ in enumerate(probabilities):
                policy[i] = probabilities[i] + T[i].dot(values)
    
            values = np.max(policy,axis=0)
            comparison = values - prev_vals
            diff =  np.max(comparison) - np.min(comparison)
            if (diff < 0.0000001) or (i > max_iter):
                return np.max(values)     

    def solve(self, is_bad_side):
        """Implement the agent"""
        good_sides = []
        for i, bad_side in enumerate(is_bad_side):
            if not bad_side:
                good_sides.append(i + 1)
        # set large enough bank roll possibility range to pass tests
        state_count = 4 * len(is_bad_side)
        
        T, R = self.set_environment(len(is_bad_side), state_count, good_sides)
        # multiply the matricies and take the sum
        r_t_probabilities = np.zeros([2, state_count])
        r_t_probabilities[0] = np.sum((R[0] * T[0]),axis=1)
        r_t_probabilities[1] = np.sum((R[1] * T[1]),axis=1)
        # define a policy for each state, roll or quit
        # print(r_t_probabilities)
        initial_value = self.determine_initial_state_value(T, r_t_probabilities, state_count)
        # print(initial_value)
        return initial_value
        


## Test cases

We have provided some test cases for you to help verify your implementation.

In [9]:
## DO NOT MODIFY THIS CODE.  This code will ensure that your submission
## will work proberly with the autograder

import unittest

class TestDieNNotebook(unittest.TestCase):
    def test_case_1(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(is_bad_side=[1, 1, 1, 0, 0, 0]),
            2.583,
            decimal=3
        )
        
    def test_case_2(self):
        agent = MDPAgent()
        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0]
            ),
            7.379,
            decimal=3
        )
        
    def test_case_3(self):
        agent = MDPAgent()

        np.testing.assert_almost_equal(
            agent.solve(
                is_bad_side=[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0]
            ),
            6.314,
            decimal=3
        )

unittest.main(argv=[''], verbosity=2, exit=False)

test_case_1 (__main__.TestDieNNotebook) ... ok
test_case_2 (__main__.TestDieNNotebook) ... ok
test_case_3 (__main__.TestDieNNotebook) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.009s

OK
